In [ ]:
import pandas as pd
import json
from collections import defaultdict
import re
from utilities import Tiempo
import dataExtraction
import statistics

#### Con este bucle obtenemos los datos sobre la inicialización de todos los niveles jugados

In [ ]:
def extraerTiempos(rawData, minTiempoInactivo):
    #HAY QUE COMPROBAR QUE minTiempoInactivo es positivo
    erInitialized = re.compile(r'\binitialized$\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    erAccessed = re.compile(r'\baccessed$\b')
    erInteracted = re.compile(r'\binteracted$\b')
    erProgressed = re.compile(r'\bprogressed$\b')

    erLevel = re.compile(r'\blevel$\b')
    erIdLevel = re.compile(r'/')
    erSeriousGame = re.compile(r'\bserious-game$\b')
    erGameObject = re.compile(r'\bgame-object$\b')
    erScreen = re.compile(r'\bscreen$\b')

    erExitGamePanel = re.compile(r'\bexit_game_panel$\b')
    erExitGamePanelCloseButton = re.compile(r'\bexit_game_panel_close_button$\b')
    
    data = defaultdict(defaultdict)
    infoNivel = defaultdict(defaultdict)
    ultimaTrazaRegistrada = defaultdict()
    tiemposInactivo = defaultdict(list)
    inicioYFinJuego = defaultdict(list)
    intentosNivel = defaultdict(defaultdict)
    anomalias = []
    ultimoNivelCompletado = defaultdict()
    
    for evento in rawData:         
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        name = evento["actor"]["name"]
        timestamp = evento["timestamp"]
        objectId = evento["object"]["id"]
        
        if erLevel.search(obj): #Si el objeto de la acción es un nivel
            levelCode = erIdLevel.split(objectId)[-1]
            if erInitialized.search(verb): #Si la acción es inicio o reinicio
                if "result" in evento: #Significa que ha iniciado el nivel desde el menu
                    if levelCode in data[name]:
                        data[name][levelCode].append({"ini" : timestamp, "fin" : None, "stars" : ""})
                    else:
                        data[name][levelCode] = [{"ini" : timestamp, "fin" : None, "stars" : ""}]
                else:
                    if levelCode in intentosNivel[name]:
                        intentosNivel[name][levelCode]["intentos"] += 1
                    else:
                        intentosNivel[name][levelCode] = {"intentos" : 1}
            elif erCompleted.search(verb):
                if evento["result"]["score"]["raw"] > 0 :
                    dataExtraction.anadirAlDiccionario(infoNivel, evento, name, levelCode)
                    if levelCode in data[name]:
                        data[name][levelCode][-1]["fin"] = timestamp
                        data[name][levelCode][-1]["stars"] = evento["result"]["score"]["raw"]
                    else: #Significa que hay un evento de finalizar que no tiene un evento de inicio
                        anomalias.append(evento)
                else:
                    if not levelCode in data[name]:
                         anomalias.append(evento) 
        
        #Algunos jugadores cierran el juego el juego con Alt+F4 o dando a la x de la ventana, esto no queda registrado
        #Para poder tener esto en cuenta habría que recorrer los datos 2 veces ya que no podemos quedarnos con el último 
        #valor antes del initialized porque los timestamps no vienen bien ordenados
        
        elif erSeriousGame.search(obj) and erInitialized.search(verb): #Nos indica que el jugador ha abierto el juego
            inicioYFinJuego[name].append({"inicio" : timestamp, "fin" : None})
            
        if erProgressed.search(verb) and erSeriousGame.search(obj):
            progresoActual = evento["result"]["extensions"]["https://w3id.org/xapi/seriousgames/extensions/progress"]
            if name in ultimoNivelCompletado:
                if progresoActual > ultimoNivelCompletado[name]["progreso"]:
                    ultimoNivelCompletado[name] = {"nivel" : erIdLevel.split(objectId)[-1], "progreso" : progresoActual}
            else:
                ultimoNivelCompletado[name] = {"nivel" : erIdLevel.split(objectId)[-1], "progreso" : progresoActual}
      
    for evento in rawData:
        name = evento["actor"]["name"]
        timestamp = evento["timestamp"]
                
        for i in range(len(inicioYFinJuego[name])):
            if (i + 1) < len(inicioYFinJuego[name]): #No estamos en la ultima partida del jugador
                if Tiempo(timestamp, inicioYFinJuego[name][i+1]["inicio"]) > Tiempo("1s"): #No es una trazas que aparece antes de iniciar el juego
                    inicioYFinJuego[name][i]["fin"] = timestamp
            else:
                inicioYFinJuego[name][i]["fin"] = timestamp
        if name in ultimaTrazaRegistrada:
                t = Tiempo(ultimaTrazaRegistrada[name], timestamp)
                if t > minTiempoInactivo:
                    tiemposInactivo[name].append(t)
                   
                    for ini in inicioYFinJuego[name]:
                        if(Tiempo(ini["inicio"], timestamp) < Tiempo("1s")):
                            del tiemposInactivo[name][-1] #Eliminamos los tiempos inactivos que sean de distintos inicios del juego
                            break
                        
        ultimaTrazaRegistrada[name] = timestamp
                    
                    
    return {"tiemposNivel" : data, "tiempoTotal" : inicioYFinJuego, "anomalias" : anomalias, "tiemposInactivo" : tiemposInactivo, "infoNivel" : infoNivel, "intentosNivel" : intentosNivel, "ultimoNivel" : ultimoNivelCompletado}              

In [ ]:
def tiempoTotalJuego(inicioYFinJuego):
    tiempoTotal = defaultdict()
    for player in inicioYFinJuego:
        for time in inicioYFinJuego[player]:
            if time["fin"] != None:
                if player in tiempoTotal:
                    tiempoTotal[player] += Tiempo(time["inicio"], time["fin"])
                else:
                    tiempoTotal[player] = Tiempo(time["inicio"], time["fin"])
    return tiempoTotal

In [ ]:
def tiempoPorNiveles(data, anomalias):
    tiemposJugados = defaultdict(defaultdict)
    for player in data:
        for level in data[player]:
            for times in data[player][level]:
                if times["fin"] != None: #Si no se aborto el intento del nivel
                    try:
                        timeDifference = Tiempo(times["ini"], times["fin"])
                        if level in tiemposJugados[player]:
                            tiemposJugados[player][level].append({"time" : timeDifference, "stars" : times["stars"]})
                        else:
                            tiemposJugados[player][level] = [{"time" : timeDifference, "stars" : times["stars"]}]
                    except ValueError: #Algunos timestamps de finalizacion estan vacios, ¿Tiene que ser asi?
                        anomalias.append({"jugador" : player,
                                  "nivel" : level,
                                  "Timestamp-Inicio" : times["ini"],
                                  "Timestamp-Fin" : times["fin"],
                                  "Descripcion:" : "No se ha podido parsear uno de los 2 timestamps"})
    return tiemposJugados

In [ ]:
def buscarTarjetas(rawData):
    data = []
    erIdLevel = re.compile(r'/')
    for evento in rawData:
        idJugador = evento["actor"]["name"]
        levelCode = erIdLevel.split(evento["object"]["id"])[-1]
        if idJugador == "bdwyj" and "result" in evento and "extensions" in evento["result"] and "level" in evento["result"]["extensions"]:
            if evento["result"]["extensions"]["level"] == "variables_9":
                data.append(evento)
        elif idJugador == "bdwyj" and levelCode == "variables_9":
            data.append(evento)
    return data

In [ ]:
def getMediaTiemposYEstrellas(data):
    tiemposYEstrellasMedios = defaultdict()
    sumasTotales = defaultdict()
    for user in data:
        for level in data[user]:
            for time in data[user][level]:
                if level in tiemposYEstrellasMedios:
                    tiemposYEstrellasMedios[level]["tiempo"] += time["time"]
                    tiemposYEstrellasMedios[level]["stars"] += time["stars"]
                    sumasTotales[level] += 1
                else:
                    tiemposYEstrellasMedios[level] = {"tiempo" : time["time"], "stars" : time["stars"]}
                    sumasTotales[level] = 1
    for level in tiemposYEstrellasMedios:
        tiemposYEstrellasMedios[level]["tiempo"] = tiemposYEstrellasMedios[level]["tiempo"]/sumasTotales[level];
        tiemposYEstrellasMedios[level]["stars"] = round(tiemposYEstrellasMedios[level]["stars"]/sumasTotales[level], 2);
    return tiemposYEstrellasMedios

In [ ]:
def getDesempenioJugadores(tiempos, tiemposMedios, porcDebajo, porcEncima, primerIntento = True):
    #tiempos -> diccionario con los tiempos de cada jugador en cada nivel
    #tiemporMedios -> media de los tiempos por nivel
    #porcDebajo -> porcentaje que se usara para filtrar a los jugadores con un tiempo igual o inferior a este
    #porcEncima -> porcentaje que se usara para filtrar a los jugadores con un tiempo igual o superior a este
    #primerIntento -> true cuando solo se quiere tener en cuenta el primer intento del jugador, false cuando no
    jugadoresPorDebajo = defaultdict(defaultdict) #Jugadores con buen desempeño, menor tiempo que media
    jugadoresPorEncima = defaultdict(defaultdict) #Jugaodre con mal desempeño, mayor tiempo que media

    for player in tiempos:
        for level in tiempos[player]:
            for time in tiempos[player][level]:
                if time["time"] <= (tiemposMedios[level]["tiempo"]*(1-porcDebajo)):
                    if level in jugadoresPorDebajo[player]:
                        jugadoresPorDebajo[player][level].append(time["time"])
                    else:
                        jugadoresPorDebajo[player][level] = [time["time"]]
                        
                elif time["time"] >= (tiemposMedios[level]["tiempo"]*(1+porcEncima)):
                    if level in jugadoresPorEncima[player]:
                        jugadoresPorEncima[player][level].append(time["time"])
                    else:
                        jugadoresPorEncima[player][level] = [time["time"]]
                        
                if primerIntento:
                    break
                    
    return jugadoresPorDebajo, jugadoresPorEncima

In [ ]:
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = None

In [ ]:
JSONFile = open('traces_Articoding_Escolapias.json')
rawData = json.load(JSONFile)
minTiempoInactivo = Tiempo("1m/30s")
result = extraerTiempos(rawData, minTiempoInactivo)
JSONFile.close()
#pd.DataFrame(result["data"])
#pd.DataFrame.from_dict(result["tiempoTotal"], orient='index').transpose()
#pd.DataFrame.from_dict(result["tiemposInactivo"], orient='index').transpose()
#pd.DataFrame(result["infoNivel"])
#pd.DataFrame(result["intentosNivel"])
pd.DataFrame.from_dict(result["ultimoNivel"], orient='index').transpose()

In [ ]:
#Numero de jugadores
len(result["intentosNivel"].keys())

In [ ]:
tiemposJ = tiempoPorNiveles(result["tiemposNivel"], result["anomalias"])
pd.DataFrame(tiemposJ)

In [ ]:
tiemposMedios = getMediaTiemposYEstrellas(tiemposJ)
pd.DataFrame(tiemposMedios)

In [ ]:
porcentajePorDebajo = 0.2
porcentajePorEncima = 0.5
jPorDebajo, jPorEncima = getDesempenioJugadores(tiemposJ, tiemposMedios, porcentajePorDebajo, porcentajePorEncima)
pd.DataFrame(jPorEncima)

In [ ]:
tiempoTJuego = tiempoTotalJuego(result["tiempoTotal"])
pd.DataFrame(tiempoTJuego, index = ["Tiempo"])
#Los tiempos del jugador knewj salen mal porque las trazas tienen mal puesto el tiempo

In [ ]:
Tiempo("2022-05-18T07:13:48.865Z", "2022-05-18T06:13:48.865Z")